<a href="https://colab.research.google.com/github/vrhughes/DS4002-Project2/blob/main/SCRIPTS/TWFE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing
import pandas as pd
import numpy as np
import re

In [2]:
# Cloning the repo
! git clone https://github.com/vrhughes/DS4002-Project2

Cloning into 'DS4002-Project2'...
remote: Enumerating objects: 108, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 108 (delta 38), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (108/108), 635.12 KiB | 3.40 MiB/s, done.
Resolving deltas: 100% (38/38), done.


In [4]:
# Reading in data
#fr_analysis = pd.read_csv('/content/DS4002-Project2/OUTPUTS/fr_analysis_df.csv')
interp = pd.read_csv('/content/DS4002-Project2/OUTPUTS/interpolated_education_dataset.csv')
UN = pd.read_csv('/content/DS4002-Project2/DATA/fertility-rate-with-projections.csv')

In [5]:
# Getting dummies
df_encoded = pd.get_dummies(interp, columns=['Country', 'Year'])

In [9]:
df_encoded.head()
print(df_encoded.columns)

Index(['Fertility_Rate', 'GDP_Per_Capita', 'Homicide_Rate',
       'Avg_Yrs_Education', 'Infant_Mortality', 'Life_Expectancy',
       'Country_Bangladesh', 'Country_Chile', 'Country_Ethiopia',
       'Country_Iran', 'Country_Latvia', 'Country_Namibia', 'Country_Nauru',
       'Country_New Zealand', 'Country_Nicaragua', 'Country_Niger',
       'Country_Norway', 'Country_Panama', 'Country_South Korea',
       'Country_United States', 'Country_Venezuela', 'Country_World',
       'Year_1980', 'Year_1981', 'Year_1982', 'Year_1983', 'Year_1984',
       'Year_1985', 'Year_1986', 'Year_1987', 'Year_1988', 'Year_1989',
       'Year_1990', 'Year_1991', 'Year_1992', 'Year_1993', 'Year_1994',
       'Year_1995', 'Year_1996', 'Year_1997', 'Year_1998', 'Year_1999',
       'Year_2000', 'Year_2001', 'Year_2002', 'Year_2003', 'Year_2004',
       'Year_2005', 'Year_2006', 'Year_2007', 'Year_2008', 'Year_2009',
       'Year_2010', 'Year_2011', 'Year_2012', 'Year_2013', 'Year_2014',
       'Year_2015', 'Y

wow that's a lot of columns!

In [15]:
pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.7 MB/s eta 0:00:00


In [16]:
# Importing for TWFE
from matplotlib import style
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import graphviz as gr
from linearmodels.panel import PanelOLS

In [23]:
# Running linear regression on the covariates and the dummies
twfe = PanelOLS.from_formula("Fertility_Rate ~ GDP_Per_Capita+Homicide_Rate+Avg_Yrs_Education+Infant_Mortality+Life_Expectancy+EntityEffects+TimeEffects", data=interp.set_index(["Country", "Year"]))

result = twfe.fit(cov_type = 'clustered', cluster_entity = True, cluster_time = True)
result.summary.tables[1]

/usr/local/lib/python3.11/dist-packages/linearmodels/panel/model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


,Parameter,Std. Err.,T-stat,P-value,Lower CI,Upper CI
GDP_Per_Capita,1.011e-05,3.136e-06,3.2240,0.0014,3.945e-06,1.627e-05
Homicide_Rate,0.0122,0.0079,1.5409,0.1241,-0.0034,0.0278
Avg_Yrs_Education,-0.4771,0.1624,-2.9385,0.0035,-0.7962,-0.1579
Infant_Mortality,0.1840,0.0920,2.0006,0.0461,0.0032,0.3648
Life_Expectancy,0.0411,0.0616,0.6675,0.5048,-0.0800,0.1622


In [25]:
# Getting coefficients and making model (mod)
mod = smf.ols(formula='Fertility_Rate ~ GDP_Per_Capita+Homicide_Rate+Avg_Yrs_Education+Infant_Mortality+Life_Expectancy', data=interp).fit()
mod.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,11.7745,0.956,12.317,0.000,9.896,13.653
GDP_Per_Capita,1.966e-05,2.1e-06,9.352,0.000,1.55e-05,2.38e-05
Homicide_Rate,0.0075,0.004,2.019,0.044,0.000,0.015
Avg_Yrs_Education,-0.2365,0.023,-10.307,0.000,-0.282,-0.191
Infant_Mortality,0.0248,0.035,0.711,0.477,-0.044,0.094
Life_Expectancy,-0.1022,0.013,-7.929,0.000,-0.127,-0.077


average years education of women decreased the fertility rates by 23%

p val low means very low chance it does not affect dependent, so low means relevant, high means irrelevantc --> homicide and infant mortality are less relevant

# STEP 2

In [ ]:
dum_yr = pd.get_dummies(interp['Year'])
dummy_it = 11.7745 + 1.966e-05*t + 0.0075*t + -0.2365*t + 0.0248*t + -0.1022*t

In [ ]:
twfe_toy = df_encoded.assign(y_hat = twfe.fittedvalues)

plt.scatter(df_encoded.Year)

Try second

X in the email is the regression variables matrix

B is beta which is the coefficients matrix


In [80]:
unit_col_name = 'Country'
time_period_col_name = 'Year'

df_panel = interp.copy()
df_panel["Country"] = df_panel["Country"].str.replace(" ", "_")

# Create dummy vars, one per country
df_dummies = pd.get_dummies(df_panel[unit_col_name], dtype=int)

# Create dummy vars, one per year
time_period_dummies = pd.get_dummies(df_panel[time_period_col_name], dtype=int, prefix=time_period_col_name)

df_panel_w_dummies = df_panel.join([df_dummies, time_period_dummies])
df_panel_w_dummies.head()

,Country,Year,Fertility_Rate,GDP_Per_Capita,Homicide_Rate,Avg_Yrs_Education,Infant_Mortality,Life_Expectancy,Bangladesh,Chile,...,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018,Year_2019,Year_2020,Year_2021,Year_2022,Year_2023
0,Bangladesh,1980,6.326,206.075714,1.672038,1.310,13.789642,52.2778,1,0,...,0,0,0,0,0,0,0,0,0,0
1,Bangladesh,1981,6.237,224.241392,1.829637,1.436,13.472392,52.6383,1,0,...,0,0,0,0,0,0,0,0,0,0
2,Bangladesh,1982,6.118,199.595945,1.987816,1.562,13.139923,53.0119,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Bangladesh,1983,5.906,184.706773,2.170414,1.688,12.788301,53.4409,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Bangladesh,1984,5.732,193.435021,2.442022,1.814,12.423361,53.8259,1,0,...,0,0,0,0,0,0,0,0,0,0


In [81]:
y_var_name = 'Fertility_Rate'
x_var_names = ['GDP_Per_Capita', 'Homicide_Rate', 'Avg_Yrs_Education', 'Infant_Mortality', 'Life_Expectancy']

In [66]:
# # work on least squares dummy var
# lsdv_expr = y_var_name + ' ~ '
# i = 0
# for X_var_name in x_var_names:
#     if i > 0:
#         lsdv_expr = lsdv_expr + ' + ' + X_var_name
#     else:
#         lsdv_expr = lsdv_expr + X_var_name
#     i = i + 1
# for dummy_name in unit_names[:-1]:
#     lsdv_expr = lsdv_expr + ' + ' + dummy_name
# for time_period in time_periods[:-1]:
#     lsdv_expr = lsdv_expr + ' + ' + time_period


# print('Regression expression for OLS with dummies=' + lsdv_expr)

Regression expression for OLS with dummies=Fertility_Rate ~ GDP_Per_Capita + Homicide_Rate + Avg_Yrs_Education + Infant_Mortality + Life_Expectancy + Bangladesh + Chile + Ethiopia + Iran + Latvia + Namibia + Nauru + New_Zealand + Nicaragua + Niger + Norway + Panama + South_Korea + United_States + Venezuela + 1980 + 1981 + 1982 + 1983 + 1984 + 1985 + 1986 + 1987 + 1988 + 1989 + 1990 + 1991 + 1992 + 1993 + 1994 + 1995 + 1996 + 1997 + 1998 + 1999 + 2000 + 2001 + 2002 + 2003 + 2004 + 2005 + 2006 + 2007 + 2008 + 2009 + 2010 + 2011 + 2012 + 2013 + 2014 + 2015 + 2016 + 2017 + 2018 + 2019 + 2020 + 2021 + 2022


In [ ]:
# # build and train LSDV on panel w dummies
# lsdv_model = smf.ols(formula=lsdv_expr, data=df_panel_w_dummies)
# lsdv_model_results = lsdv_model.fit()
# print(lsdv_model_results.summary())

In [82]:
import pandas as pd
import statsmodels.formula.api as smf

# Construct the formula with the correct dummy variable names
lsdv_expr = y_var_name + ' ~ '
i = 0
for X_var_name in x_var_names:
    if i > 0:
        lsdv_expr = lsdv_expr + ' + ' + X_var_name
    else:
        lsdv_expr = lsdv_expr + X_var_name
    i = i + 1

# Include country dummies (excluding the last one for reference)
for dummy_name in df_dummies.columns[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name

# Include year dummies (excluding the last one for reference)
for dummy_name in time_period_dummies.columns[:-1]:
    lsdv_expr = lsdv_expr + ' + ' + dummy_name  # No need for C() anymore

print('Regression expression for OLS with dummies=' + lsdv_expr)

# build and train LSDV on panel w dummies
lsdv_model = smf.ols(formula=lsdv_expr, data=df_panel_w_dummies)
lsdv_model_results = lsdv_model.fit()
print(lsdv_model_results.summary())

Regression expression for OLS with dummies=Fertility_Rate ~ GDP_Per_Capita + Homicide_Rate + Avg_Yrs_Education + Infant_Mortality + Life_Expectancy + Bangladesh + Chile + Ethiopia + Iran + Latvia + Namibia + Nauru + New_Zealand + Nicaragua + Niger + Norway + Panama + South_Korea + United_States + Venezuela + Year_1980 + Year_1981 + Year_1982 + Year_1983 + Year_1984 + Year_1985 + Year_1986 + Year_1987 + Year_1988 + Year_1989 + Year_1990 + Year_1991 + Year_1992 + Year_1993 + Year_1994 + Year_1995 + Year_1996 + Year_1997 + Year_1998 + Year_1999 + Year_2000 + Year_2001 + Year_2002 + Year_2003 + Year_2004 + Year_2005 + Year_2006 + Year_2007 + Year_2008 + Year_2009 + Year_2010 + Year_2011 + Year_2012 + Year_2013 + Year_2014 + Year_2015 + Year_2016 + Year_2017 + Year_2018 + Year_2019 + Year_2020 + Year_2021 + Year_2022
                            OLS Regression Results                            
Dep. Variable:         Fertility_Rate   R-squared:                       0.962
Model:            